In [ ]:
#Instalo vaderSentiment para el analisis de sentimientos
!pip install vaderSentiment

In [ ]:
#Instalo langdetect para detectar el idioma
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=9c9131b88474e4ab13cddf0a5b801e4941fe0d1474a8c95a2a5b2915de1f8597
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
#Importo las librerias necesarias para la carga y tratamiento de los datos

import pandas as pd
from pandas import json_normalize
import json
import ast
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from langdetect import detect
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re


In [ ]:
# Leo el archivo de datos de juegos de steam

archivo1=[]

with open("output_steam_games.json","r", encoding= 'utf-8') as filejson:

    for line in filejson.readlines():

        archivo1.append(json.loads(line))


steam_games=pd.DataFrame(archivo1)


In [ ]:
#Leo el archivo de reseñas de usuarios

archivo2=[]

with open("australian_user_reviews.json","r", encoding= 'utf-8') as filejson:

    for line in filejson.readlines():

        fila=ast.literal_eval(line)

        archivo2.append(fila)


user_reviews=pd.DataFrame(archivo2)


In [ ]:
#Leo el archivo de items adquiridos por los usuarios

archivo3=[]

with open("australian_users_items.json","r", encoding= 'utf-8') as filejson:

    for line in filejson.readlines():

        archivo3.append(ast.literal_eval(line))


user_items=pd.DataFrame(archivo3)

In [ ]:
#Se borran los registros que no tengan un id de referencia

steam_games.dropna(inplace=True,subset='id')

#Elimino columnas que no son necesarias para el sistema de recomendación

steam_games= steam_games.drop(['reviews_url','price','early_access','title'], axis=1)

#Se llenan los espacios vacios con 'No data'

steam_games.fillna('No data',inplace=True)

#Se reinician los indices

steam_games.reset_index(inplace=True,drop=True)

In [ ]:
#Se convierten las listas en conjuntos para próximas consultas de las columnas genres,tags y specs.
def convertir_en_conjunto(data):

  if data=='No data':
    return {'No data'}

  else:

    return set(data)

steam_games['genres'],steam_games['tags'],steam_games['specs']=steam_games['genres'].apply(convertir_en_conjunto),steam_games['tags'].apply(convertir_en_conjunto),steam_games['specs'].apply(convertir_en_conjunto)

In [ ]:
"""Esta función year() se crea para tratar los datos de la columna release_date, dejando únicamente
el año de lanzamiento de cada producto, en caso de no especificarlo se reemplaza por 'No data'"""


def year(string):
    # Patrón de año (cuatro dígitos)
    patron_year = re.compile(r'\b\d{4}\b')

    # Buscar coincidencias en la cadena
    coincidencias = patron_year.findall(string)

    # Devolver el primer año encontrado o 'No data' si no hay coincidencias
    return int(coincidencias[0]) if coincidencias else 'No data'


#Se aplica la función year sobre la columna release_date

steam_games['release_date']=steam_games['release_date'].apply(year)


In [ ]:
#Se cambian el nombre de la columna id por item_id para que coincida con el Dataframe de user_items
steam_games= steam_games.rename(columns={'id': 'item_id'})

#Uno los generos de genres y tags
steam_games['genres'] = steam_games.apply(lambda row: row['genres'].union(row['tags']), axis=1)

# Eliminar la columna 'tags'
steam_games.drop(columns=['tags'], inplace=True)

In [ ]:


# Función para desanidar una fila
def desanidar(fila):

    fila = pd.Series(fila, index=['user_id', 'items_count', 'steam_id', 'user_url', 'items'])

    if fila['items'] != []:

        items_desplegados = json_normalize(fila['items'])
        fila_duplicada = pd.concat([fila.to_frame().T] * items_desplegados.shape[0], ignore_index=True)
        df_desanidado = pd.concat([fila_duplicada, items_desplegados], axis=1)
        df_desanidado.dropna(subset=['item_id'], inplace=True)

        return df_desanidado
    else:
        return fila

# Aplicar la función desanidar sobre todo el archivo user_items y concatenar los resultados
df_auxiliar_desanidado = pd.concat([desanidar(fila) for fila in user_items.itertuples(index=False)], ignore_index=True)

# Eliminar las columnas 'items' y 'playtime_2weeks'
df_auxiliar_desanidado = df_auxiliar_desanidado.drop(['items', 'playtime_2weeks'], axis=1)

# Rellenar datos faltantes con 'No data'
df_auxiliar_desanidado.fillna('No data', inplace=True)

# Reemplazar user_items por el df_auxiliar completamente desanidado
user_items = df_auxiliar_desanidado


In [ ]:
#Se eliminan usuarios que tiene 'No data'
user_items=user_items[user_items['user_id']!='No data']

#Se elimina una columna que quedo despues del proceso de desanidado
user_items = user_items.drop([0], axis=1)



In [ ]:
#Se agregan datos de steam games que coinciden con el item_id
user_items=pd.merge(user_items,steam_games,on='item_id',how='left')

#Se eligen los datos originales agregando la columna de generos y la fecha de lanzamiento
user_items=user_items.loc[:,['user_id', 'items_count', 'steam_id', 'user_url', 'item_id',
       'item_name', 'playtime_forever','genres','release_date']]



In [ ]:
#Los datos de genero faltante se rellenan con {'No data'}
user_items.fillna('No data',inplace=True)

In [ ]:
def release_date_search(nombre_juego):
    # Formatear el nombre del juego para la URL de Wikipedia
    nombre_juego = nombre_juego.replace(" ", "_")
    url = f'https://en.wikipedia.org/wiki/{nombre_juego}'

    # Realizar la solicitud HTTP
    respuesta = requests.get(url)

    # Verificar si la solicitud fue exitosa
    if respuesta.status_code == 200:
        # Utilizar BeautifulSoup para analizar el HTML
        soup = BeautifulSoup(respuesta.text, 'html.parser')

        # Encontrar la sección que contiene la descripción del juego
        descripcion_seccion = soup.find('div', {'id': 'mw-content-text'})

        # Extraer el texto de la descripción
        descripcion_texto = descripcion_seccion.get_text() if descripcion_seccion else ""

        # Utilizar expresiones regulares para encontrar el primer año en el texto
        anios_encontrados = re.findall(r'\b\d{4}\b', descripcion_texto)

        # Devolver el primer año encontrado o un mensaje si no se encuentra ninguno
        if anios_encontrados:
            return int(anios_encontrados[0])
        else:
            return 'No data'
    else:
        return 'No data'





In [ ]:
juegos_sin_fecha=user_items[user_items['release_date']=='No data']
juegos_sin_fecha=juegos_sin_fecha.drop_duplicates(subset='item_id')
juegos_sin_fecha.reset_index(inplace=True,drop=True)


In [ ]:


#Hallo los datos que faltan

lista_datos_faltantes= juegos_sin_fecha['item_name'].apply(lambda x: release_date_search(x))
lista_datos_faltantes=lista_datos_faltantes.rename('release_date')


In [ ]:

juegos_sin_fecha=juegos_sin_fecha.drop(columns=['release_date'])
juegos_sin_fecha=pd.concat([juegos_sin_fecha,lista_datos_faltantes],axis=1)

In [ ]:
juegos_sin_fecha=juegos_sin_fecha.loc[:,['item_id','release_date']]

In [ ]:
user_items_renovado = pd.merge(user_items, juegos_sin_fecha, on='item_id', how='left')


In [ ]:
#Reemplazo los datos que me faltaban"

user_items_renovado[user_items_renovado['release_date_x']=='No data']['release_date_x']= user_items_renovado[user_items_renovado['release_date_x']=='No data']['release_date_y']



<ipython-input-22-2de906160af3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_items_renovado[user_items_renovado['release_date_x']=='No data']['release_date_x']= user_items_renovado[user_items_renovado['release_date_x']=='No data']['release_date_y']


In [ ]:
user_items_renovado=user_items_renovado.drop(columns=['release_date_y'])
user_items_renovado=user_items_renovado.rename(columns={'release_date_x': 'release_date'})
user_items=user_items_renovado

In [ ]:
#Hago un DataFrame auxiliar
df_resultado=pd.DataFrame({'funny':[],'posted':[], 'last_edited':[],'item_id':[], 'helpful':[],'recommend':[],'review':[]})

rango=user_reviews.shape[0]

#Desanido la columna  reviews


for nfila in range(0,rango):

  df_desanidado = json_normalize(user_reviews['reviews'][nfila])
  reviews=pd.Series({'user_id':user_reviews['user_id'][nfila]})
  reviews2= reviews

  for i in range(1,df_desanidado.shape[0]):

    reviews=pd.concat([reviews,reviews2],axis=0,ignore_index=True)


  if user_reviews['reviews'][nfila]!=[]:
    df_result=pd.concat([reviews,df_desanidado],axis=1)

  else:

    df_result=reviews

  df_resultado=pd.concat([df_resultado,df_result],axis=0,ignore_index=True)



df_resultado = df_resultado.rename(columns={0: 'user_id'})

user_reviews2=pd.merge(user_reviews,df_resultado, on='user_id', how='inner')

user_reviews2= user_reviews2.drop(['reviews','funny','last_edited','helpful','user_url'], axis=1)


<ipython-input-26-4d0de0ac5bb3>:27: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df_resultado=pd.concat([df_resultado,df_result],axis=0,ignore_index=True)


In [ ]:
user_reviews2['review']=user_reviews2['review'].fillna(1)

user_reviews2.dropna(inplace=True,subset='item_id')

user_reviews2.reset_index(inplace=True,drop=True)

In [ ]:
def es_ingles(texto):
    try:
        idioma = detect(texto)
        return idioma == 'en'
    except:
        return False


def etiquetar_sentimiento(texto):

  if es_ingles(texto):

    analyzer = SentimentIntensityAnalyzer()
    compound_score = analyzer.polarity_scores(texto)['compound']

    # Establecer umbrales según tu preferencia
    if compound_score >= 0.05:
        return 2
    elif compound_score <= -0.05:
        return 0
    else:
        return 1

  else:


    # Creo un objeto TextBlob
    blob = TextBlob(texto)

    # Obtengo la polaridad del sentimiento (-1 a 1)
    polaridad = blob.sentiment.polarity

    # Clasifico el sentimiento
    if polaridad > 0.05:
        return 2
    elif polaridad < -0.05:
        return 0
    else:
        return 1


user_reviews3=[]

for reseña in user_reviews2['review']:

  user_reviews3.append(etiquetar_sentimiento(reseña))

user_reviews3=pd.Series(user_reviews3,name="review_sentimental")

In [ ]:
user_reviews2['review']=user_reviews3

user_reviews=user_reviews2

In [ ]:
recommend=user_reviews['recommend']

for i in range(0,recommend.shape[0]):

  if recommend.iloc[i]==1.0:

    recommend.iloc[i]='True'


<ipython-input-30-8f5b2cc3f310>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend.iloc[i]='True'


In [ ]:
user_reviews['recommend']=recommend

In [ ]:
user_reviews['posted']=user_reviews['posted'].astype(str)
user_reviews['posted']=user_reviews['posted'].apply(year)


In [ ]:
user_items=user_items[user_items['playtime_forever']!=0]

In [ ]:
steam_games.drop(columns=['url','publisher'], inplace=True)

In [ ]:
user_items.drop(columns=['steam_id'], inplace=True)

<ipython-input-35-0a7bc3f1c0a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_items.drop(columns=['steam_id'], inplace=True)


In [ ]:
user_items.drop(columns=['user_url','items_count',], inplace=True)

<ipython-input-36-195abbbcae29>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_items.drop(columns=['user_url','items_count',], inplace=True)


In [ ]:
# Obtén el tamaño del DataFrame
total_rows = len(user_items)

# Calcula los índices para la división
idx1 = int(total_rows * 0.33)  # Primer tercio
idx2 = int(total_rows * 0.66)  # Segundo tercio

# Divide el DataFrame en tres partes
user_items1 = user_items.iloc[:idx1]
user_items2 = user_items.iloc[idx1:idx2]
user_items3 = user_items.iloc[idx2:]

In [ ]:
user_items1.to_csv('user_items1.csv', index=False, sep=';')
user_items2.to_csv('user_items2.csv', index=False, sep=';')
user_items3.to_csv('user_items3.csv', index=False, sep=';')
steam_games.to_csv('steam_games.csv', index=False, sep=';')
user_reviews.to_csv('user_reviews.csv', index=False, sep=';')



In [ ]:
from google.colab import files

files.download('user_items1.csv')
files.download('user_items2.csv')
files.download('user_items3.csv')
files.download('user_reviews.csv')
files.download('steam_games.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>